In [ ]:
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import numpy as np
import math
import itertools
import scipy.special as ss
ň = None
from collections import namedtuple
rs2 = math.sqrt(.5)

In [ ]:
# "Fake multipoles" consist of a list constituent dipoles 
# (their powers/directions and relative placements)
# One radiating "particle" is represented by its center position, a list of FakeMultipoles, and a corresponding list of excitation coefficients
FMPart = namedtuple("FMPart", ["dipole", "offset"])
class FakeMultipole:
    def __init__(self, partlist):
        self.parts = partlist
    
    def __repr__(self):
        return "FakeMultipole("+repr(self.parts)+")"
        
    @staticmethod
    def DipX():
        return FakeMultipole((FMPart((1.,0.),(0.,0.)),))
    
    @staticmethod
    def DipY():
        return FakeMultipole((FMPart((0.,1.),(0.,0.)),))
    
    @staticmethod
    def QuadXYPar(r):
        '''
         🡣
        🡠 🡢
         🡡
        '''
        return FakeMultipole((
            FMPart(( 1., 0.),(  r, 0.)),
            FMPart((-1., 0.),( -r, 0.)),
            FMPart(( 0., 1.),( 0., -r)),
            FMPart(( 0.,-1.),( 0.,  r)),
        ))
    
    @staticmethod
    def QuadXYDiag(r):
        '''
        🡤 🡧
        🡥 🡦
        '''
        return FakeMultipole((
            FMPart((-rs2, rs2),(-rs2*r, rs2*r)),
            FMPart((-rs2,-rs2),( rs2*r, rs2*r)),
            FMPart(( rs2, rs2),(-rs2*r,-rs2*r)),
            FMPart(( rs2,-rs2),( rs2*r,-rs2*r)),
        ))

def FloatTuple(x): # normal form to be used as key
    return tuple(float(xx) for xx in x)

def MultipoleSource(fakemultipoles, facs):
    '''
    TODO description
    returns two numpy arrays of size (N, 2): offsets, dipoles
    offsets are relative to "particle center", so one can add an overall particle position to that
    dipoles can be multiplied e.g. by an overall phase
    '''
    # find unique offsets
    offsets = {FloatTuple(p.offset) for fakemultipole in fakemultipoles for p in fakemultipole.parts }
    facs = np.array(facs)
    # evaluate the overall dipole for each offset
    dipoles = {k : np.zeros(facs.shape[:-1] + (2,), dtype=complex) for k in offsets}
    for fac, fakemultipole in zip(np.moveaxis(facs, -1, 0), fakemultipoles):
        for dipole, offset in fakemultipole.parts:
            dipoles[FloatTuple(offset)] += fac[...,ň] * np.array(dipole)
    # "flatten" the result
    res_offsets = []
    res_dipoles = []
    for offset in sorted(dipoles.keys()):
        dipole = dipoles[offset]
        res_offsets.append(offset)
        res_dipoles.append(dipole)
    return np.array(res_offsets), np.moveaxis(np.array(res_dipoles), 0, -2)

def project_onto_angle(a, proj_angle=None, axis=-1):
    '''
    project plane vectors into a direction specified by proj_angle (relative to x axis)
    '''
    a = np.array(a)
    if proj_angle is None:
        return a
    pv = np.array((math.cos(proj_angle),math.sin(proj_angle)), dtype=float)
    proj = np.tensordot(a, pv, axes=(axis,-1))
    proj = np.moveaxis((proj[...,ň] * pv), -1, axis)
    proj*=(abs(proj)>1e-13) # filter away roundoff errors
    return proj
            

In [ ]:
o, d = MultipoleSource([FakeMultipole.QuadXYPar(0.2), FakeMultipole.DipX(), FakeMultipole.DipY()],[[.5,.5,.5],[4, 1, 1]])
#MultipoleSource([FakeMultipole.QuadXYPar(0.2), FakeMultipole.DipX(), FakeMultipole.DipY()],[4, 1, 1])
d.shape

In [ ]:
# El. field in Airy disk behaves as J(x)/x where x = r п 

def intensity_predict(
    NA = 1.1,
    nx = 60,
    ny = 60,
    ssl = 60/6, # square edge length in image pixels
    offset = (0.0000,0.0000), # offset of (some) A-site in pixels , TODO probably remove
    kmultiplier = 0.5, # wave vector length relative to the |Γ1 - Γ0| length (0.5 for X1 "beams")
    wangle = np.pi*(1/2+np.arange(4)/2), # wave directions w.r.t. X-axis
    phases = (0,0,0,0),
    # This will now be more complicated: facs array is now twodimensional: 
    # first dimension corresponds to k-vector index (i.e. wave propagation direction),
    # second dimension corresponds to multipole index (as in `basis`)
    # also, the polarisation (the `up` direction) is now not rotated inside the function according to the wave direction
    # but must be provided "by the caller" via facs
    basis = [FakeMultipole.DipX(), FakeMultipole.DipY(), FakeMultipole.QuadXYPar(.11), FakeMultipole.QuadXYDiag(.11)],
    facs = #((-1,-1,1,1)),
        np.array((.5,0,.5,0))[:,ň] * np.array([0,1,0,0])[ň,:],
    #wi = 0, #TODO THIS WILL GO AWAY
    gphase=1,
    # xx transversal=True, # whether the (in-plane) el. field direction is perpendicular to the (in-plane part of) wave vector

    xjfactor = 1,
):
    
    offset = np.array(offset)
    gphase = np.array(gphase)
    gphase = np.ravel(gphase)
    phases = np.array(phases, dtype=float) + gphase
    facs = np.array(facs)
    
    # set up direct lattice
    # finite lattice size
    n1 = math.ceil(nx / ssl * 2) * 2
    n2 = math.ceil(ny / ssl * 2) * 2
    # basis vectors
    b1 = np.array((ssl,0))
    b2 = np.array((0,ssl))
    # positions
    sites = np.arange(-n1,n1)[:,ň,ň] * b1[ň,ň,:] + np.arange(-n2,n2)[ň,:,ň] * b2[ň,ň,:]
    sites = np.reshape(sites,(np.prod(sites.shape[:-1]), sites.shape[-1])) # shape == ()
    sites_s = sites + offset
    
    
    # set up wave vectors and phases at sites
    klen = 2 * np.pi / ssl * kmultiplier #in-plane wavevector length
    #wlen = 2*np.pi/Xlen
    # XX upangle = wangle + pp*transversal # in-plane wave field polarisation direction
    wavevector = klen * np.stack((np.cos(wangle),np.sin(wangle)), axis=-1)
    # XX #this is for dipole polarisations, TODO generalise for multipoles
    # XX up = np.stack((np.cos(upangle),np.sin(upangle)), axis=-1) 
    # removing lattice sites, but why? Probably more relevant for non-square lattices
    if False:
        cn = 4
        sites = sites[(sites_s[...,0] <= nx+cn*ssl) 
                       *(sites_s[...,0] >= -cn*ssl) 
                       *(sites_s[...,1] <= ny+cn*ssl)
                       *(sites_s[...,1] >= -cn*ssl)]
        sites_s = sites + offset

    phase = np.tensordot(wavevector, sites, axes=(-1,-1))-phases[:,ň] # ([wi,/x/]*[site,/x/])-[wi, ň] ~ [wi,site]
    
    soffsets, sdipoles = MultipoleSource(basis, facs) # [microsite1, /x/], [wi, microsite1, /x/]
    dipsites = (sites_s[:,ň,:] + soffsets[ň,:,:]) # [site, microsite1, /x/]
    dipoles = np.exp(1j*phase[:,:,ň,ň]) * sdipoles[:,ň,:,:] # [wi, site, microsite1, /x/] 
    
    sites_s = dipsites.reshape((-1,2)) # [microsite, /x/]
    #print(sites_s.shape, dipoles.shape, dipoles.shape[:-3] + sites_s.shape)
    wave = dipoles.reshape(dipoles.shape[:-3] + sites_s.shape).real # [wi, microsite, /x/]
    
    # sum over wi (wave direction index) to get the final superposition
    wave = np.sum(wave, axis=-3)
        
    
    my = np.arange(ny)
    mx = np.arange(nx)
    xj = np.sqrt((sites_s[:,0,ň,ň]-mx[ň,ň,:])**2 + (sites_s[:,1,ň,ň]-my[ň,:,ň])**2) * xjfactor #arbitrary factor, FIXME
    
    xj[np.isnan(xj)]=0
    xj *= 2*np.pi*NA/klen
    U = ss.j1(xj)/xj # FIXME possible division by zero; site,my,mx
    U = U[:,ň,:,:] * wave[:,:,ň,ň] # [site,x,my,mx]
    U = np.moveaxis(U, (-4,-3), (-2,-1)) # [my, mx, site, x]
    U = np.sum(U,axis=-2) #  [my, mx, x]
    
    return U, sites_s, wave,

def plot_intensity_predict(ax, U, dipsites=None, dipoles=None,
        proj_angle=None, # polarisation filtering: project the result vector fields onto this direction (None for no filter)
        rs_intensity_cmap = "gist_gray", sitecolor = 'grey',arrowcolor = 'red',proj_arrowcolor='red',
        #show_dipoles = True, # show the original 
        ascale=6, # arrow scale
        show_axes = False,):
    scale_units = 'width'
    dipplot=None
    U = project_onto_angle(U, proj_angle, axis=-1)
    intensity = U[...,0]**2+U[...,1]**2 #[(?),my,mn]
    # sum over the "trailing" dimensions in the beginning; this is to simulate non-coherent pulses
    if intensity.ndim > 2:
        intensity = np.sum(intensity, tuple(range(intensity.ndim-2)))
    iplot = ax.imshow(intensity, cmap=rs_intensity_cmap)
    ax.set_xlim(-0.5,intensity.shape[0]-0.5)
    ax.set_ylim(-0.5,intensity.shape[1]-0.5)
    if dipsites is not None:
        #ax.scatter(sites_s[:,0], sites_s[:,1], marker='o', edgecolors=sitecolor, facecolors='none', s=ssl**2/nx/ny*2500) 
        if dipoles is not None:
            dipoles = project_onto_angle(dipoles, proj_angle)
            dipplot = ax.quiver(dipsites[:,0],dipsites[:,1],dipoles[:,0],dipoles[:,1],color=arrowcolor,headwidth=3, scale_units = scale_units, scale=ascale, pivot='mid')
            # pcax = ax.quiver(sites_s[:,0],sites_s[:,1],wave_proj[:,0],wave_proj[:,1],color=proj_arrowcolor,headwidth=3, scale_units = scale_units, scale=ascale, pivot='mid', width=.01)      
    if not show_axes:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
    return iplot, dipplot

In [ ]:
angles = [
    ("unpolarised", None),
    ("0 ° (x)", 0),
    ("45 °", np.pi/4),
    ("90 ° (y)", np.pi/2),
]
mpfactors = [
    ("y dipoles", (0,1,0,0)),
    ("x dipoles", (1,0,0,0)),
    ("xy axis-parallel quadrupoles", (0,0,1,0)),
    ("xy axis-diagonal quadrupoles", (0,0,0,1)),
]
kfactors = [
    ("y propagation", (1,0,0,0))
]
xjfactor = 0.05

In [ ]:
nrows = len(mpfactors)*len(kfactors)
f, axes = plt.subplots(nrows, len(angles), figsize=(10*len(angles),10*nrows))
for i, ((klabel, kf), (mplabel, mf)) in enumerate(itertools.product(kfactors, mpfactors)): # row
    axs = axes[i]
    facs = np.array(kf)[:,ň] * np.array(mf)[ň,:]
    U, sites, dipoles = intensity_predict(facs = facs, xjfactor = xjfactor)
    for ai, (alabel, angle) in enumerate(angles): #column
        ax = axs[ai]
        iplot,dipplot=plot_intensity_predict(ax, U, sites, dipoles, proj_angle = angle)
        ax.set_title(mplabel + ", " + klabel + ", " + alabel)
plt.savefig("squarelat_ypropagation_separate.pdf")

In [ ]:
angles = [
    ("unpolarised", None),
    ("0 ° (x)", 0),
    ("22.5 °", np.pi/8),
    ("45 °", np.pi/4),
    ("67.5 °", 3*np.pi/8),
    ("90 ° (y)", np.pi/2),
]
nrows = len(mpfactors)*len(kfactors)
f, axes = plt.subplots(nrows, len(angles), figsize=(10*len(angles),10*nrows))
for i, ((klabel, kf), (mplabel, mf)) in enumerate(itertools.product(kfactors, mpfactors)): # row
    axs = axes[i]
    facs = np.array(kf)[:,ň] * np.array(mf)[ň,:]
    U, sites, dipoles = intensity_predict(facs = facs, xjfactor = xjfactor)
    for ai, (alabel, angle) in enumerate(angles): #column
        ax = axs[ai]
        iplot,dipplot=plot_intensity_predict(ax, U, sites, dipoles, proj_angle = angle)
        ax.set_title(mplabel + ", " + klabel + ", " + alabel)
plt.savefig("squarelat_ypropagation_separate_big.pdf")

In [ ]:
np.random.seed(42)
angles = [
    ("unpolarised", None),
    ("0 ° (x)", 0),
    ("22.5 °", np.pi/8),
    ("45 °", np.pi/4),
    ("67.5 °", 3*np.pi/8),
    ("90 ° (y)", np.pi/2),
]
nsamples = 20
mplabel, mf = "xy axis-parallel quadrupole", np.array((0,0,1,0))

kfactors = np.random.random((nsamples, 4))
phasess = np.random.random((nsamples, 4))*2*np.pi - np.pi

Us = []
nrows = nsamples + 1
f, axes = plt.subplots(nrows, len(angles), figsize=(10*len(angles),10*nrows))
for i, (kf, phases) in enumerate(zip(kfactors, phasess)): # row
    axs = axes[i]
    facs = np.array(kf)[:,ň] * np.array(mf)[ň,:]
    U, sites, dipoles = intensity_predict(facs = facs, xjfactor = xjfactor, phases=phases)
    Us.append(U)
    for ai, (alabel, angle) in enumerate(angles): #column
        ax = axs[ai]
        iplot,dipplot=plot_intensity_predict(ax, U, sites, dipoles, proj_angle = angle)
        ax.set_title(mplabel + ", " + "random directions and phases" + ", " + alabel)
# Plot the sum:
i = nrows-1
axs = axes[i]
for ai, (alabel, angle) in enumerate(angles): #column
    ax = axs[ai]
    iplot,dipplot=plot_intensity_predict(ax, Us, #sites, dipoles, 
                                         proj_angle = angle)
    ax.set_title(mplabel + ", " + "sum over random directions and phases" + ", " + alabel)
plt.savefig("squarelat_randompropagation_xyquad_parallel_big.pdf")

In [ ]:
np.random.seed(42)
angles = [
    ("unpolarised", None),
    ("0 ° (x)", 0),
    ("22.5 °", np.pi/8),
    ("45 °", np.pi/4),
    ("67.5 °", 3*np.pi/8),
    ("90 ° (y)", np.pi/2),
]
nsamples = 20
mplabel, mf = "xy axis-diagonal quadrupole", np.array((0,0,0,1))

kfactors = np.random.random((nsamples, 4))
phasess = np.random.random((nsamples, 4))*2*np.pi - np.pi

Us = []
nrows = nsamples + 1
f, axes = plt.subplots(nrows, len(angles), figsize=(10*len(angles),10*nrows))
for i, (kf, phases) in enumerate(zip(kfactors, phasess)): # row
    axs = axes[i]
    facs = np.array(kf)[:,ň] * np.array(mf)[ň,:]
    U, sites, dipoles = intensity_predict(facs = facs, xjfactor = xjfactor, phases=phases)
    Us.append(U)
    for ai, (alabel, angle) in enumerate(angles): #column
        ax = axs[ai]
        iplot,dipplot=plot_intensity_predict(ax, U, sites, dipoles, proj_angle = angle)
        ax.set_title(mplabel + ", " + "random directions and phases" + ", " + alabel)
# Plot the sum:
i = nrows-1
axs = axes[i]
for ai, (alabel, angle) in enumerate(angles): #column
    ax = axs[ai]
    iplot,dipplot=plot_intensity_predict(ax, Us, #sites, dipoles, 
                                         proj_angle = angle)
    ax.set_title(mplabel + ", " + "sum over random directions and phases" + ", " + alabel)
plt.savefig("squarelat_randompropagation_xyquad_diagonal_big.pdf")

In [ ]:
offsets, dips = MultipoleSource([FakeMultipole.QuadXYPar(0.2), FakeMultipole.QuadXYDiag(0.2), FakeMultipole.DipX(), FakeMultipole.DipY()],[4, 4, 0, 0])
plt.quiver(offsets[:,0], offsets[:,1], dips[:,0], dips[:,1], pivot='mid')